In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import time
import pandas as pd

class CteeScraper:
    def __init__(self):
        # 設定 Chrome 選項
        self.chrome_options = webdriver.ChromeOptions()
        # self.chrome_options.add_argument('--headless')  # 無頭模式，不開啟瀏覽器視窗
        self.chrome_options.add_argument('--disable-gpu')
        self.chrome_options.add_argument('--no-sandbox')
        self.chrome_options.add_argument('--disable-dev-shm-usage')
        
        # 初始化瀏覽器
        self.driver = webdriver.Chrome(
            service=Service(ChromeDriverManager().install()),
            options=self.chrome_options
        )
        
    def get_news(self):
        try:
            # 訪問網頁
            self.driver.get('https://www.ctee.com.tw/livenews/stock')
            time.sleep(2)  # 等待頁面載入
            
            # 等待新聞列表載入
            news_elements = WebDriverWait(self.driver, 4).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.newslist__card'))
            )
            
            news_data = []
            # 解析新聞資料
            for news in news_elements:
                try:
                    title = news.find_element(By.CSS_SELECTOR, 'h3.news-title').text
                    time_date = news.find_element(By.CSS_SELECTOR, '.news-date').text
                    time_text = news.find_element(By.CSS_SELECTOR, '.news-time').text
                    # link = news.find_element(By.TAG_NAME, 'a').get_attribute('href')
                    link = news.find_element(By.CSS_SELECTOR, 'h3.news-title a').get_attribute('href')
                    # link = WebDriverWait(news, 10).until(
                    #     EC.presence_of_element_located((By.CSS_SELECTOR, 'a'))
                    # ).get_attribute('href')

                    news_data.append({
                        'date': time_date,
                        'time': time_text,
                        'title': title,
                        'link': link
                    })
                except Exception as e:
                    print(f"解析單則新聞時發生錯誤: {str(e)}")
                    continue
            
            # 轉換成 DataFrame
            df = pd.DataFrame(news_data)
            # 儲存成 CSV
            df.to_csv('ctee_news.csv', index=False, encoding='utf-8-sig')
            return df
            
        except Exception as e:
            print(f"爬取過程發生錯誤: {str(e)}")
            return None
            
        finally:
            self.driver.quit()

# 執行爬蟲
if __name__ == "__main__":
    scraper = CteeScraper()
    news_df = scraper.get_news()
    if news_df is not None:
        print("爬取成功！")
        print(news_df)

爬取成功！
     time                                         title  \
0   16:45                     冠軍半年獲利近5千萬！群益股王爭霸戰健康財富全到手   
1   15:36                    《通網股》遠傳雙軸轉型成效卓著　獲「金恆獎」金獎肯定   
2   15:20        《通網股》撘載聯發科天璣9400 台灣大專案0元入手OPPO Find X8   
3   14:56                  《通網股》電子發票升級！中華電獲「雲端物聯網創新獎」肯定   
4   14:46               《百貨股》統一超推「7-ELEVEN饗喫鍋火鍋專區」搶冒煙商機   
5   13:25                 《航運股》華航「皮卡丘彩繪機CI」飛進2024高雄冬季旅展   
6   11:38                     《櫃買市場》昕奇雲端、立誠光電送件申請股票櫃檯買賣   
7   10:44                  《航運股》長榮航再度蟬聯「體育推手獎」 唯一獲獎國籍航空   
8   10:40                  如何抱股跨年？4大作帳行情年底拚績效 專家點名9檔成主流   
9   09:57                《台北股市》川普2.0擊敗消費旺季利多 11月台股跌幅逾2%   
10  09:34                       台股轉空了嗎？兩大黑天鵝襲來 土洋法人同步保守   
11  08:30         比特幣、無人機、BBU...明年錢往哪裡走？川普交易2.0重點 專家全解析   
12  08:11             台股探底 要轉戰金融股？不是富邦金、國泰金 億元教授1關鍵選中信金   
13  07:17  AI遜、台積失守千元 下個明星產業在哪？蔡明彰曝台股止跌看1關鍵 「這產業」三大咖都在推   
14  03:00                       土洋對作 法人衝作帳行情 點名14檔年底最會漲   
15  03:00                             期貨研討會 抓住新變局下